In [37]:
import numpy as np

def debug():
    print(' kernel: ' + KERNEL)
    print('      d: ' + str(d))
    print('      D: ' + str(D))
    print('epsilon: ' + str(epsilon))
    print('**************\n')

In [66]:
KERNEL  = 'G'      # G => Gaussian, L => Laplacian, C => Cauchy
d       = 28 * 28  # dimension of input data
epsilon = 0.1      # acceptable error for kernel

def compute_D(d, epsilon):
    return int(0.1 * d * np.log(epsilon ** (-2))) # * (epsilon ** (-2))

D       = compute_D(d, epsilon)

# w is a d-dimensional vector
def pdf(w):
    if KERNEL == 'G':
        # (2pi)^(-D/2) e^(-0.5 ||w||_2^2)
        return ((2 * np.pi) ** (-D / 2)) * np.exp(-0.5 * (np.linalg.norm(w) ** 2))

# Monte-Carlo Rejection Sampling (https://en.wikipedia.org/wiki/Rejection_sampling)
def sample(interval):
    # computes max value of PDF; specific to kernel
    pdf_max = 1
    if KERNEL == 'G':
        pdf_max = ((2 * np.pi) ** (-D / 2))

    while True:
        w = np.random.rand(1) * (interval[1] - interval[0]) + interval[0]
        y = np.random.rand(1) * pdf_max

        if y <= pdf(w):
            return w

# Random Features for Large-Scale Kernel Machines (Rahimi & Recht, 2007)
# https://papers.nips.cc/paper/3182-random-features-for-large-scale-kernel-machines.pdf
def z(x):
    # normalize random vector to sampled norm
    randoms = [np.random.rand(d) for _ in range(D)]
    w = np.asarray([sample((-10, 10)) * (random / np.linalg.norm(random)) for random in randoms])
    b = np.asarray([np.random.uniform(0, 2 * np.pi) for _ in range(D)])

    return np.sqrt(2 / D) * np.cos(np.dot(w, x) + b)

# uses random features to approximate the Gaussian kernel
def K(x, y):
    return np.dot(z(x).T, z(y))